In [12]:
import streamlit as st
import streamlit.components.v1 as components
import csv
import string
import os
import pickle
import pandas as pd
from datetime import datetime as dt
from base64 import b64encode
#import proximitypyhash as pph
#import pygeohash as gh
from streamlit_folium import st_folium
from streamlit_folium import folium_static
import folium
import numpy as np

from agile.filtering import query_location, query_date, query_adid, query_node
from agile.mapping import data_map 
from agile.locations import locations_of_interest
from agile.people import colocation
from agile.prediction import double_cluster, get_top_N_clusters, haversine
from agile.utils.tag import find_all_nearby_nodes
from agile.utils.geocode import reverse_geocode
from agile.utils.files import find, random_line, save, random_name
from agile.utils.dataframes import modify_and_sort_columns
from agile.profile import Profile
from agile.samsreport import Report
from agile.centrality import compute_top_centrality
from agile.overview import adid_value_counts

from streamlit_option_menu import option_menu
import pygeohash as gh

In [13]:
with open('saved_data/saved_df.pkl', 'rb') as file:
    df = pickle.load(file)

In [14]:
adid = '21191f02-c141-e3f6-901e-f0c9222be423'
num_clusters = 4

cluster_data = double_cluster(adid, df)
loi_data = get_top_N_clusters(cluster_data, num_clusters)
loi_data_i = pd.DataFrame()
loi_data

loi_data['iter'] = [0] * len(loi_data)

In [15]:
def filter_close_coordinates(df, threshold_distance=0.13):
    new_df = pd.DataFrame(columns=df.columns)

    for i, row1 in df.iterrows():
        keep_row = True
        for _, row2 in new_df.iterrows():
            distance = haversine(row1['latitude'], row1['longitude'], row2['latitude'], row2['longitude'])
            if distance < threshold_distance:
                keep_row = False
                break
        if keep_row:
            new_df = new_df.append(row1, ignore_index=True)

    return new_df

In [16]:
cluster_data = pd.DataFrame()
loi_data = pd.DataFrame()
loi_data_i = pd.DataFrame()

for i in range(1,5):
    print(i)
    cluster_data = double_cluster(adid, df)
    loi_data_i = get_top_N_clusters(cluster_data, i)
    loi_data_i['i'] = [i] * len(loi_data_i)
    loi_data_i = filter_close_coordinates(loi_data_i, threshold_distance=.7)
    loi_data = pd.concat([loi_data, loi_data_i])
    
for i in range(1,25,3):
    for j in range(73,0,-4):
        print(i,j)
        loi_data = pd.concat([loi_data,locations_of_interest(df, adid, i, j)]).reset_index(drop=True)        

loi_data = loi_data.drop_duplicates()        
    
loi_data

1

C:\Users\m245370\AppData\Local\Temp\ipykernel_15776\2294503313.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(row1, ignore_index=True)


2


C:\Users\m245370\AppData\Local\Temp\ipykernel_15776\2294503313.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(row1, ignore_index=True)


3


C:\Users\m245370\AppData\Local\Temp\ipykernel_15776\2294503313.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(row1, ignore_index=True)


4


C:\Users\m245370\AppData\Local\Temp\ipykernel_15776\2294503313.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(row1, ignore_index=True)


1 73
1 69
1 65
1 61
1 57
1 53
1 49
1 45
1 41
1 37
1 33
1 29
1 25
1 21
1 17
1 13
1 9
1 5
1 1
4 73
4 69
4 65
4 61
4 57
4 53
4 49
4 45
4 41
4 37
4 33
4 29
4 25
4 21
4 17
4 13
4 9
4 5
4 1
7 73
7 69
7 65
7 61
7 57
7 53
7 49
7 45
7 41
7 37
7 33
7 29
7 25
7 21
7 17
7 13
7 9
7 5
7 1
10 73
10 69
10 65
10 61
10 57
10 53
10 49
10 45
10 41
10 37
10 33
10 29
10 25
10 21
10 17
10 13
10 9
10 5
10 1
13 73
13 69
13 65
13 61
13 57
13 53
13 49
13 45
13 41
13 37
13 33
13 29
13 25
13 21
13 17
13 13
13 9
13 5
13 1
16 73
16 69
16 65
16 61
16 57
16 53
16 49
16 45
16 41
16 37
16 33
16 29
16 25
16 21
16 17
16 13
16 9
16 5
16 1
19 73
19 69
19 65
19 61
19 57
19 53
19 49
19 45
19 41
19 37
19 33
19 29
19 25
19 21
19 17
19 13
19 9
19 5
19 1
22 73
22 69
22 65
22 61
22 57
22 53
22 49
22 45
22 41
22 37
22 33
22 29
22 25
22 21
22 17
22 13
22 9
22 5
22 1


,latitude,longitude,datetime,geohash,advertiser_id,label,i
0,46.192594,-118.989033,2018-04-08 08:34:51.500,c25qu34g6e,21191f02-c141-e3f6-901e-f0c9222be423,300,1
1,46.168194,-118.953881,2018-04-08 08:42:25.000,c25qthsmjx,21191f02-c141-e3f6-901e-f0c9222be423,297,2
2,46.219753,-119.028130,2018-04-08 08:27:44.000,c25qgqn3ed,21191f02-c141-e3f6-901e-f0c9222be423,205,2
3,46.138270,-118.784817,2018-04-08 09:13:45.500,c25w2zpwch,21191f02-c141-e3f6-901e-f0c9222be423,300,3
4,46.137824,-118.739311,2018-04-08 09:21:22.500,c25w6p0sm9,21191f02-c141-e3f6-901e-f0c9222be423,71,3
5,46.263104,-119.166829,2018-04-08 08:02:19.000,c25r0mcq5e,21191f02-c141-e3f6-901e-f0c9222be423,158,3
6,46.263075,-119.173469,2018-04-08 08:01:23.000,c25r0jutby,21191f02-c141-e3f6-901e-f0c9222be423,298,4
7,46.252196,-119.138353,2018-04-08 08:07:41.000,c25r0gyne4,21191f02-c141-e3f6-901e-f0c9222be423,150,4
8,46.257741,-119.156291,2018-04-08 08:04:08.500,c25r0scnc5,21191f02-c141-e3f6-901e-f0c9222be423,287,4
9,46.204964,-119.006874,2018-04-08 08:31:26.500,c25qggmg6v,21191f02-c141-e3f6-901e-f0c9222be423,284,4


In [17]:
import pandas as pd
from datetime import datetime, timedelta

def colocation(data: pd.DataFrame, lois: pd.DataFrame, duration: int) -> pd.DataFrame:
    """
    Finds devices that were detected within a certain distance and time range of a given set of locations of interest.

    Args:
        data: A DataFrame containing all of the data.
        lois: A DataFrame of locations of interest for a single advertiser ID.
        duration: An int representing the time range in hours to match timeframes of datapoints.

    Returns:
        A DataFrame containing co-located devices.
    """
    relevant_features = ['geohash', 'datetime', 'latitude', 'longitude', 'advertiser_id']
    data = data[relevant_features].sort_values(by="datetime").reset_index(drop=True)

    filtered = data[data.geohash.isin(lois.geohash.unique())]
    
    try:
        filtered = filtered[filtered.advertiser_id != lois.advertiser_id.iloc[0]]
    except:
        pass
    
    print(filtered)
    
    search_time = timedelta(hours=duration)
    
    def time_filter(row):
        loi_filtered = lois[lois.geohash == row.geohash]
        loi_dates = pd.to_datetime(loi_filtered.datetime)
        filtered_time = pd.to_datetime(row.datetime)
        within_timerange = (filtered_time > (loi_dates - search_time)) & (filtered_time < (loi_dates + search_time))
        row['remove'] = not within_timerange.any()
        return row

    filtered = filtered.apply(time_filter, axis=1)
    data_out = None
    if 'remove' in filtered.columns:
        data_out = filtered.loc[filtered.remove == False].drop(columns=['remove'])

    return data_out


In [18]:
loi_data = filter_close_coordinates(loi_data, threshold_distance=.5)
loi_data

C:\Users\m245370\AppData\Local\Temp\ipykernel_15776\2294503313.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(row1, ignore_index=True)


,latitude,longitude,datetime,geohash,advertiser_id,label,i
0,46.192594,-118.989033,2018-04-08 08:34:51.500,c25qu34g6e,21191f02-c141-e3f6-901e-f0c9222be423,300,1
1,46.168194,-118.953881,2018-04-08 08:42:25.000,c25qthsmjx,21191f02-c141-e3f6-901e-f0c9222be423,297,2
2,46.219753,-119.028130,2018-04-08 08:27:44.000,c25qgqn3ed,21191f02-c141-e3f6-901e-f0c9222be423,205,2
3,46.138270,-118.784817,2018-04-08 09:13:45.500,c25w2zpwch,21191f02-c141-e3f6-901e-f0c9222be423,300,3
4,46.137824,-118.739311,2018-04-08 09:21:22.500,c25w6p0sm9,21191f02-c141-e3f6-901e-f0c9222be423,71,3
5,46.263104,-119.166829,2018-04-08 08:02:19.000,c25r0mcq5e,21191f02-c141-e3f6-901e-f0c9222be423,158,3
6,46.263075,-119.173469,2018-04-08 08:01:23.000,c25r0jutby,21191f02-c141-e3f6-901e-f0c9222be423,298,4
7,46.252196,-119.138353,2018-04-08 08:07:41.000,c25r0gyne4,21191f02-c141-e3f6-901e-f0c9222be423,150,4
8,46.257741,-119.156291,2018-04-08 08:04:08.500,c25r0scnc5,21191f02-c141-e3f6-901e-f0c9222be423,287,4
9,46.204964,-119.006874,2018-04-08 08:31:26.500,c25qggmg6v,21191f02-c141-e3f6-901e-f0c9222be423,284,4


In [19]:
# Create a Folium map centered around the mean coordinates
map_center = [loi_data['latitude'].mean(), loi_data['longitude'].mean()]
folium_map = folium.Map(location=map_center, zoom_start=5)

# Add markers for each row in the DataFrame
for index, row in loi_data.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']]).add_to(folium_map)
    
folium_map

In [20]:
colocation = colocation(df, loi_data, duration=12)
colocation

            geohash            datetime   latitude   longitude  \
180509   c25r50813j 2018-04-08 09:05:17  46.233459 -119.047799   
459898   c25qggmg6v 2018-04-08 11:53:20  46.204965 -119.006874   
888955   c25r50813j 2018-04-08 16:00:12  46.233460 -119.047800   
949525   c25r0gyne4 2018-04-08 16:38:09  46.252198 -119.138351   
1004926  c25qmzm97m 2018-04-08 17:53:06  46.138704 -118.919300   

                                advertiser_id  
180509   40479742-b498-0805-0955-9e8760308341  
459898   45018a74-387c-5c62-4d06-94757e634009  
888955   301cda66-80b3-8694-05e1-6615e1572a39  
949525   12797791-6b15-5485-d439-6304159800c7  
1004926  02d7a139-0908-6344-9185-69355c6f41b6  


,geohash,datetime,latitude,longitude,advertiser_id
180509,c25r50813j,2018-04-08 09:05:17,46.233459,-119.047799,40479742-b498-0805-0955-9e8760308341
459898,c25qggmg6v,2018-04-08 11:53:20,46.204965,-119.006874,45018a74-387c-5c62-4d06-94757e634009
888955,c25r50813j,2018-04-08 16:00:12,46.233460,-119.047800,301cda66-80b3-8694-05e1-6615e1572a39
949525,c25r0gyne4,2018-04-08 16:38:09,46.252198,-119.138351,12797791-6b15-5485-d439-6304159800c7
1004926,c25qmzm97m,2018-04-08 17:53:06,46.138704,-118.919300,02d7a139-0908-6344-9185-69355c6f41b6


In [23]:
loi_data = pd.DataFrame()

for i in range(1,25,3):
    for j in range(71,0,-10):
        print(i,j)
        loi_data = pd.concat([loi_data,locations_of_interest(df, adid, i, j)]).reset_index(drop=True)
        
loi_data

1 71
1 61
1 51
1 41
1 31
1 21
1 11
1 1
4 71
4 61
4 51
4 41
4 31
4 21
4 11
4 1
7 71
7 61
7 51
7 41
7 31
7 21
7 11
7 1
10 71
10 61
10 51
10 41
10 31
10 21
10 11
10 1
13 71
13 61
13 51
13 41
13 31
13 21
13 11
13 1
16 71
16 61
16 51
16 41
16 31
16 21
16 11
16 1
19 71
19 61
19 51
19 41
19 31
19 21
19 11
19 1
22 71
22 61
22 51
22 41
22 31
22 21
22 11
22 1


,geohash,datetime,latitude,longitude,advertiser_id
0,c25qmzm97m,2018-04-08 10:51:57,46.138704,-118.919306,21191f02-c141-e3f6-901e-f0c9222be423
1,c25pr8t7h2,2018-04-08 14:24:29,46.277680,-119.194261,21191f02-c141-e3f6-901e-f0c9222be423
2,c25r50813j,2018-04-08 14:50:22,46.233462,-119.047800,21191f02-c141-e3f6-901e-f0c9222be423
3,c25qmzm97m,2018-04-08 10:51:57,46.138704,-118.919306,21191f02-c141-e3f6-901e-f0c9222be423
4,c25pr8t7h2,2018-04-08 14:24:29,46.277680,-119.194261,21191f02-c141-e3f6-901e-f0c9222be423
5,c25r50813j,2018-04-08 14:50:22,46.233462,-119.047800,21191f02-c141-e3f6-901e-f0c9222be423
6,c25qmzm97m,2018-04-08 10:51:57,46.138704,-118.919306,21191f02-c141-e3f6-901e-f0c9222be423
7,c25pr8t7h2,2018-04-08 14:24:29,46.277680,-119.194261,21191f02-c141-e3f6-901e-f0c9222be423
8,c25r50813j,2018-04-08 14:50:22,46.233462,-119.047800,21191f02-c141-e3f6-901e-f0c9222be423
9,c25qmzm97m,2018-04-08 10:51:57,46.138704,-118.919306,21191f02-c141-e3f6-901e-f0c9222be423


In [22]:
loi_data.drop_duplicates()

,geohash,datetime,latitude,longitude,advertiser_id
0,c25pr8t7h2,2018-04-08 14:24:29,46.277680,-119.194261,21191f02-c141-e3f6-901e-f0c9222be423
1,c25r50813j,2018-04-08 14:50:22,46.233462,-119.047800,21191f02-c141-e3f6-901e-f0c9222be423
2,c25qmzm97m,2018-04-08 10:51:57,46.138704,-118.919306,21191f02-c141-e3f6-901e-f0c9222be423


In [31]:
colo = pd.DataFrame(colocation['advertiser_id'].unique(),columns=['Colocated IDs'])
colo

,Colocated IDs
0,40479742-b498-0805-0955-9e8760308341
1,45018a74-387c-5c62-4d06-94757e634009
2,301cda66-80b3-8694-05e1-6615e1572a39
3,12797791-6b15-5485-d439-6304159800c7
4,02d7a139-0908-6344-9185-69355c6f41b6


In [29]:
coloc_ids = pd.DataFrame(columns=['Colocated IDs'])
coloc_ids

,Colocated IDs


In [33]:
coloc_ids.append(colo)

C:\Users\m245370\AppData\Local\Temp\ipykernel_15776\1359737043.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  coloc_ids.append(colo)


,Colocated IDs
0,40479742-b498-0805-0955-9e8760308341
1,45018a74-387c-5c62-4d06-94757e634009
2,301cda66-80b3-8694-05e1-6615e1572a39
3,12797791-6b15-5485-d439-6304159800c7
4,02d7a139-0908-6344-9185-69355c6f41b6
